In [3]:
import pandas as pd
import glob

# Clean Packet Capture

In [2]:
# List of all csv in a dir
csv_files = glob.glob('captures1_v2/clean/*.csv')

clean_df = pd.DataFrame()

# append to combined dataframe
for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    clean_df = pd.concat([clean_df, df])

clean_df['AttackName'] = 'Clean'

clean_df.head()


,No.,Time,SrcIP,DstIP,Protocol,Length,Info,SrcMAC,SrcMACResolved,SrcOUIResolved,...,DstOUResolved,DstPort,DstMACResolved.1,SYNFlag,ACKFlag,ProtocolType,EpochTime,RelativeTime,TimeDelta,AttackName
0,1,0.000000,172.27.224.70,172.27.224.250,Modbus/TCP,66,"Query: Trans: 0; Unit: 1, Func: 3: ...",00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,Modbus/TCP,1.536430e+09,0.000000,0.000000,Clean
1,2,0.004884,172.27.224.250,172.27.224.70,Modbus/TCP,85,"Response: Trans: 0; Unit: 1, Func: 3: ...",00:80:f4:09:51:3b,Telemech_09:51:3b,Telemechanique Electrique,...,"VMware, Inc.",49205.0,VMware_9d:9e:9e,Not set,Set,Modbus/TCP,1.536430e+09,0.004884,0.004884,Clean
2,3,0.037927,172.27.224.251,172.27.224.250,TCP,60,58944 > 502 [ACK] Seq=1 Ack=1 Win=2036 Len=0,48:5b:39:64:40:79,ASUSTekC_64:40:79,ASUSTek COMPUTER INC.,...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,TCP,1.536430e+09,0.037927,0.033043,Clean
3,4,0.217459,172.27.224.70,172.27.224.250,TCP,60,49205 > 502 [ACK] Seq=13 Ack=32 Win=65206 Len=0,00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,TCP,1.536430e+09,0.217459,0.179532,Clean
4,5,0.311723,172.27.224.70,172.27.224.250,Modbus/TCP,66,"Query: Trans: 0; Unit: 1, Func: 3: ...",00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,Modbus/TCP,1.536430e+09,0.311723,0.094264,Clean


In [3]:
# Check for missing values
missing_values = clean_df.isnull().sum()
print(missing_values)

No.                     0
Time                    0
SrcIP                   0
DstIP                   0
Protocol                0
Length                  0
Info                    0
SrcMAC                  0
SrcMACResolved          0
SrcOUIResolved          0
SrcPort             40771
SequenceNumber      46224
SrcOUI                  0
DstMAC                  0
DstMACResolved          0
DstOUI                  0
DstOUResolved       35895
DstPort             40771
DstMACResolved.1        0
SYNFlag             46224
ACKFlag             46224
ProtocolType            0
EpochTime               0
RelativeTime            0
TimeDelta               0
AttackName              0
dtype: int64


In [5]:
# Count the number of packets where SrcPort is not applicable (null)
srcport_not_applicable = clean_df['SrcPort'].isnull().sum()
print(f"Number of packets where SrcPort is not applicable (null): {srcport_not_applicable}")

# Count the number of packets where DstPort is not applicable (null)
dstport_not_applicable = clean_df['DstPort'].isnull().sum()
print(f"Number of packets where DstPort is not applicable (null): {dstport_not_applicable}")

# Count the number of packets where SrcPort should be applicable but is null
srcport_missing = mitm_df[(mitm_df['Protocol'] == 'TCP') | (mitm_df['Protocol'] == 'UDP')]['SrcPort'].isnull().sum()
print(f"Number of packets where SrcPort should be applicable but is null: {srcport_missing}")

# Count the number of packets where DstPort should be applicable but is null
dstport_missing = clean_df[(clean_df['Protocol'] == 'TCP') | (clean_df['Protocol'] == 'UDP')]['DstPort'].isnull().sum()
print(f"Number of packets where DstPort should be applicable but is null: {dstport_missing}")


Number of packets where SrcPort is not applicable (null): 40771
Number of packets where DstPort is not applicable (null): 40771
Number of packets where SrcPort should be applicable but is null: 0
Number of packets where DstPort should be applicable but is null: 0


In [6]:
# Count the number of packets where SequenceNumber is not applicable (null)
seqnum_not_applicable = clean_df['SequenceNumber'].isnull().sum()
print(f"Number of packets where SequenceNumber is not applicable (null): {seqnum_not_applicable}")

# Count the number of packets where SequenceNumber should be applicable but is null
seqnum_missing = clean_df[(clean_df['Protocol'] == 'TCP')]['SequenceNumber'].isnull().sum()
print(f"Number of packets where SequenceNumber should be applicable but is null: {seqnum_missing}")


Number of packets where SequenceNumber is not applicable (null): 46224
Number of packets where SequenceNumber should be applicable but is null: 0


In [8]:
# Count the number of packets where SYNFlag is not applicable (null)
synflag_not_applicable = clean_df['SYNFlag'].isnull().sum()
print(f"Number of packets where SYNFlag is not applicable (null): {synflag_not_applicable}")

# Count the number of packets where ACKFlag is not applicable (null)
ackflag_not_applicable = clean_df['ACKFlag'].isnull().sum()
print(f"Number of packets where ACKFlag is not applicable (null): {ackflag_not_applicable}")

# Count the number of packets where SYNFlag should be applicable but is null
synflag_missing = clean_df[(clean_df['Protocol'] == 'TCP')]['SYNFlag'].isnull().sum()
print(f"Number of packets where SYNFlag should be applicable but is null: {synflag_missing}")

# Count the number of packets where ACKFlag should be applicable but is null
ackflag_missing = clean_df[(clean_df['Protocol'] == 'TCP')]['ACKFlag'].isnull().sum()
print(f"Number of packets where ACKFlag should be applicable but is null: {ackflag_missing}")


Number of packets where SYNFlag is not applicable (null): 46224
Number of packets where ACKFlag is not applicable (null): 46224
Number of packets where SYNFlag should be applicable but is null: 0
Number of packets where ACKFlag should be applicable but is null: 0


In [9]:
# Count the number of packets where DstOUResolved is not applicable (null)
dstou_not_applicable = clean_df['DstOUResolved'].isnull().sum()
print(f"Number of packets where DstOUResolved is not applicable (null): {dstou_not_applicable}")

# Count the number of packets where DstOUResolved should be applicable but is null
dstou_missing = clean_df['DstOUResolved'].isnull().sum()  # No specific protocol condition since DstOUResolved can be applicable to various protocols
print(f"Number of packets where DstOUResolved should be applicable but is null: {dstou_missing}")


Number of packets where DstOUResolved is not applicable (null): 35895
Number of packets where DstOUResolved should be applicable but is null: 35895


In [10]:
# Count the number of packets where DstOUResolved is not applicable (null)
dstou_not_applicable = clean_df['DstOUResolved'].isnull().sum()
print(f"Number of packets where DstOUResolved is not applicable (null): {dstou_not_applicable}")

# Count the number of packets where DstOUResolved should be applicable but is null
dstou_missing = clean_df[((clean_df['Protocol'] == 'TCP') | (clean_df['Protocol'] == 'UDP')) & clean_df['DstOUResolved'].isnull()].shape[0]
print(f"Number of packets where DstOUResolved should be applicable but is null: {dstou_missing}")


Number of packets where DstOUResolved is not applicable (null): 35895
Number of packets where DstOUResolved should be applicable but is null: 1286


In [11]:

# 1. Handling missing values in SrcPort and DstPort
clean_df['SrcPort'].fillna(-1, inplace=True)  # Filling missing SrcPort with -1
clean_df['DstPort'].fillna(-1, inplace=True)  # Filling missing DstPort with -1

# 2. Handling missing values in SequenceNumber
# Assuming TCP packets have SequenceNumber and using median imputation
median_seqnum = clean_df[clean_df['Protocol'] == 'TCP']['SequenceNumber'].median()
clean_df['SequenceNumber'].fillna(median_seqnum, inplace=True)

# 3. Handling missing values in DstOUResolved
clean_df['DstOUResolved'].fillna('Unknown', inplace=True)  # Filling missing DstOUResolved with 'Unknown'

# 4. Handling missing values in SYNFlag and ACKFlag
clean_df['SYNFlag'].fillna(False, inplace=True)  # Filling missing SYNFlag with False
clean_df['ACKFlag'].fillna(False, inplace=True)  # Filling missing ACKFlag with False

In [12]:
# Check for missing values
missing_values = clean_df.isnull().sum()
print(missing_values)

No.                 0
Time                0
SrcIP               0
DstIP               0
Protocol            0
Length              0
Info                0
SrcMAC              0
SrcMACResolved      0
SrcOUIResolved      0
SrcPort             0
SequenceNumber      0
SrcOUI              0
DstMAC              0
DstMACResolved      0
DstOUI              0
DstOUResolved       0
DstPort             0
DstMACResolved.1    0
SYNFlag             0
ACKFlag             0
ProtocolType        0
EpochTime           0
RelativeTime        0
TimeDelta           0
AttackName          0
dtype: int64


In [ ]:
# Convert EpochTime to datetime format and create 'date_time' column
clean_df['date_time'] = pd.to_datetime(df['EpochTime'], unit='s')

# Man in the Middle Attack

In [13]:
csv_files = glob.glob('captures1_v2/mitm/*.csv')

mitm_df = pd.DataFrame()

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    mitm_df = pd.concat([mitm_df, df])

mitm_df['AttackName'] = 'mitm'

mitm_df.head()

,No.,Time,SrcIP,DstIP,Protocol,Length,Info,SrcMAC,SrcMACResolved,SrcOUIResolved,...,DstOUResolved,DstPort,DstMACResolved.1,SYNFlag,ACKFlag,ProtocolType,EpochTime,RelativeTime,TimeDelta,AttackName
0,1,0.000000,172.27.224.70,172.27.224.250,TCP,60,49499 > 502 [ACK] Seq=1 Ack=1 Win=65144 Len=0,00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,TCP,1.535051e+09,0.000000,0.000000,mitm
1,2,0.094452,172.27.224.70,172.27.224.250,Modbus/TCP,66,"Query: Trans: 0; Unit: 1, Func: 3: ...",00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,Modbus/TCP,1.535051e+09,0.094452,0.094452,mitm
2,3,0.097765,172.27.224.250,172.27.224.70,Modbus/TCP,85,"Response: Trans: 0; Unit: 1, Func: 3: ...",00:80:f4:09:51:3b,Telemech_09:51:3b,Telemechanique Electrique,...,"VMware, Inc.",49499.0,VMware_9d:9e:9e,Not set,Set,Modbus/TCP,1.535051e+09,0.097765,0.003313,mitm
3,4,0.311944,172.27.224.70,172.27.224.250,TCP,60,49499 > 502 [ACK] Seq=13 Ack=32 Win=65113 Len=0,00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,TCP,1.535051e+09,0.311944,0.214179,mitm
4,5,0.316011,172.27.224.251,172.27.224.250,TCP,60,"54093 > 502 [FIN, ACK] Seq=1 Ack=1 Win=2036 ...",48:5b:39:64:40:79,ASUSTekC_64:40:79,ASUSTek COMPUTER INC.,...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,TCP,1.535051e+09,0.316011,0.004067,mitm


In [14]:
# Check for missing values
missing_values = mitm_df.isnull().sum()
print(missing_values)

No.                      0
Time                     0
SrcIP                    0
DstIP                    0
Protocol                 0
Length                   0
Info                     0
SrcMAC                   0
SrcMACResolved           0
SrcOUIResolved           0
SrcPort             142616
SequenceNumber      155786
SrcOUI                   0
DstMAC                   0
DstMACResolved           0
DstOUI                   0
DstOUResolved       112079
DstPort             142616
DstMACResolved.1         0
SYNFlag             155786
ACKFlag             155786
ProtocolType             0
EpochTime                0
RelativeTime             0
TimeDelta                0
AttackName               0
dtype: int64


In [19]:
# 1. Handling missing values in SrcPort and DstPort
mitm_df['SrcPort'].fillna(-1, inplace=True)  # Filling missing SrcPort with -1
mitm_df['DstPort'].fillna(-1, inplace=True)  # Filling missing DstPort with -1

# 2. Handling missing values in SequenceNumber
# Assuming TCP packets have SequenceNumber and using median imputation
median_seqnum = mitm_df[mitm_df['Protocol'] == 'TCP']['SequenceNumber'].median()
mitm_df['SequenceNumber'].fillna(median_seqnum, inplace=True)

# 3. Handling missing values in DstOUResolved
mitm_df['DstOUResolved'].fillna('Unknown', inplace=True)  # Filling missing DstOUResolved with 'Unknown'

# 4. Handling missing values in SYNFlag and ACKFlag
mitm_df['SYNFlag'].fillna(False, inplace=True)  # Filling missing SYNFlag with False
mitm_df['ACKFlag'].fillna(False, inplace=True)  # Filling missing ACKFlag with False

In [20]:
# Check for missing values
missing_values = mitm_df.isnull().sum()
print(missing_values)

No.                 0
Time                0
SrcIP               0
DstIP               0
Protocol            0
Length              0
Info                0
SrcMAC              0
SrcMACResolved      0
SrcOUIResolved      0
SrcPort             0
SequenceNumber      0
SrcOUI              0
DstMAC              0
DstMACResolved      0
DstOUI              0
DstOUResolved       0
DstPort             0
DstMACResolved.1    0
SYNFlag             0
ACKFlag             0
ProtocolType        0
EpochTime           0
RelativeTime        0
TimeDelta           0
AttackName          0
dtype: int64


In [ ]:
# Convert EpochTime to datetime format and create 'date_time' column
mitm_df['date_time'] = pd.to_datetime(df['EpochTime'], unit='s')

# modbusQuery2Flooding

In [12]:
csv_files = glob.glob('captures1_v2/modbusQuery2Flooding/*.csv')

modbusQuery2Flooding_df = pd.DataFrame()

for csv_file in csv_files:
    try:
        df = pd.read_csv(csv_file)
        modbusQuery2Flooding_df = pd.concat([modbusQuery2Flooding_df, df])
    except Exception as e:
        print(f"Failed to read {csv_file}: {e}")

modbusQuery2Flooding_df['AttackName'] = 'modbusQuery2Flooding'

modbusQuery2Flooding_df.head()

Failed to read captures1_v2/modbusQuery2Flooding/eth2dump-modbusQuery2Flooding30m-6h_1.csv: Error tokenizing data. C error: Expected 25 fields in line 1258221, saw 38

Failed to read captures1_v2/modbusQuery2Flooding/eth2dump-modbusQuery2Flooding30m-1h_1.csv: 'utf-8' codec can't decode byte 0x86 in position 246153: invalid start byte


,No.,Time,SrcIP,DstIP,Protocol,Length,Info,SrcMAC,SrcMACResolved,SrcOUIResolved,...,DstOUResolved,DstPort,DstMACResolved.1,SYNFlag,ACKFlag,ProtocolType,EpochTime,RelativeTime,TimeDelta,AttackName
0,1,0.000000,172.27.224.70,172.27.224.250,TCP,60,50260 > 502 [ACK] Seq=1 Ack=1 Win=64927 Len=0,00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,TCP,1.527001e+09,0.000000,0.000000,modbusQuery2Flooding
1,2,0.095001,172.27.224.70,172.27.224.250,Modbus/TCP,66,"Query: Trans: 0; Unit: 1, Func: 3: ...",00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,Modbus/TCP,1.527001e+09,0.095001,0.095001,modbusQuery2Flooding
2,3,0.101238,172.27.224.250,172.27.224.70,Modbus/TCP,85,"Response: Trans: 0; Unit: 1, Func: 3: ...",00:80:f4:09:51:3b,Telemech_09:51:3b,Telemechanique Electrique,...,"VMware, Inc.",50260.0,VMware_9d:9e:9e,Not set,Set,Modbus/TCP,1.527001e+09,0.101238,0.006237,modbusQuery2Flooding
3,4,0.304811,HewlettP_8e:40:b3,Spanning-tree-(for-bridges)_00,STP,64,RST. Root = 32768/0/00:18:6e:d7:8a:c0 Cost = ...,d0:7e:28:8e:40:b3,HewlettP_8e:40:b3,Hewlett Packard,...,NaN,NaN,Spanning-tree-(for-bridges)_00,NaN,NaN,STP,1.527001e+09,0.304811,0.203573,modbusQuery2Flooding
4,5,0.311951,172.27.224.70,172.27.224.250,TCP,60,50260 > 502 [ACK] Seq=13 Ack=32 Win=64896 Len=0,00:0c:29:9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,TCP,1.527001e+09,0.311951,0.007140,modbusQuery2Flooding


In [13]:
# Check for missing values
missing_values = modbusQuery2Flooding_df.isnull().sum()
print(missing_values)

No.                      0
Time                     0
SrcIP                    0
DstIP                    0
Protocol                 0
Length                   0
Info                     0
SrcMAC                   0
SrcMACResolved           0
SrcOUIResolved           0
SrcPort             342203
SequenceNumber      378862
SrcOUI                   0
DstMAC                   0
DstMACResolved           0
DstOUI                   0
DstOUResolved       283669
DstPort             342203
DstMACResolved.1         0
SYNFlag             378862
ACKFlag             378862
ProtocolType             0
EpochTime                0
RelativeTime             0
TimeDelta                0
AttackName               0
dtype: int64


In [14]:
# 1. Handling missing values in SrcPort and DstPort
modbusQuery2Flooding_df['SrcPort'].fillna(-1, inplace=True)  # Filling missing SrcPort with -1
modbusQuery2Flooding_df['DstPort'].fillna(-1, inplace=True)  # Filling missing DstPort with -1

# 2. Handling missing values in SequenceNumber
# Assuming TCP packets have SequenceNumber and using median imputation
median_seqnum = modbusQuery2Flooding_df[modbusQuery2Flooding_df['Protocol'] == 'TCP']['SequenceNumber'].median()
modbusQuery2Flooding_df['SequenceNumber'].fillna(median_seqnum, inplace=True)

# 3. Handling missing values in DstOUResolved
modbusQuery2Flooding_df['DstOUResolved'].fillna('Unknown', inplace=True)  # Filling missing DstOUResolved with 'Unknown'

# 4. Handling missing values in SYNFlag and ACKFlag
modbusQuery2Flooding_df['SYNFlag'].fillna(False, inplace=True)  # Filling missing SYNFlag with False
modbusQuery2Flooding_df['ACKFlag'].fillna(False, inplace=True)  # Filling missing ACKFlag with False

In [15]:
# Check for missing values
missing_values = modbusQuery2Flooding_df.isnull().sum()
print(missing_values)

No.                 0
Time                0
SrcIP               0
DstIP               0
Protocol            0
Length              0
Info                0
SrcMAC              0
SrcMACResolved      0
SrcOUIResolved      0
SrcPort             0
SequenceNumber      0
SrcOUI              0
DstMAC              0
DstMACResolved      0
DstOUI              0
DstOUResolved       0
DstPort             0
DstMACResolved.1    0
SYNFlag             0
ACKFlag             0
ProtocolType        0
EpochTime           0
RelativeTime        0
TimeDelta           0
AttackName          0
dtype: int64


In [ ]:
# Convert EpochTime to datetime format and create 'date_time' column
modbusQuery2Flooding_df['date_time'] = pd.to_datetime(df['EpochTime'], unit='s')

# modbusQueryFlooding

In [4]:
csv_files = glob.glob('captures1_v2/modbusQueryFlooding/*.csv')

modbusQueryFlooding_df = pd.DataFrame()

for csv_file in csv_files:
    try:
        df = pd.read_csv(csv_file)
        modbusQueryFlooding_df = pd.concat([tcpSYNFlood_df, df])
    except Exception as e:
        print(f"Failed to read {csv_file}: {e}")

modbusQueryFlooding_df['AttackName'] = 'modbusQueryFlooding'

modbusQueryFlooding_df.head()

,No.,Time,SrcIP,DstIP,Protocol,Length,Info,SrcMAC,SrcMACResolved,SrcOUIResolved,...,DstOUResolved,DstPort,DstMACResolved.1,SYNFlag,ACKFlag,ProtocolType,EpochTime,TimeDelta,RelativeTime,AttackName
0,1,0.000000,172.27.224.70,172.27.224.250,Modbus/TCP,66,"Query: Trans: 0; Unit: 1, Func: 3: ...",VMware_9d:9e:9e,VMware_9d:9e:9e,"VMware, Inc.",...,Telemecanique Electrique,502.0,Telemecaniqu_09:51:3b,Not set,Set,Modbus/TCP,1.527317e+09,Not set,0.000000,modbusQueryFlooding
1,2,0.000013,172.27.224.251,172.27.224.250,TCP,60,"54084 > 502 [FIN, ACK] Seq=1 Ack=1 Win=2036 ...",ASUSTekCOMPU_64:40:79,ASUSTekCOMPU_64:40:79,ASUSTek COMPUTER INC.,...,Telemecanique Electrique,502.0,Telemecaniqu_09:51:3b,Not set,Set,TCP,1.527317e+09,Not set,0.000013,modbusQueryFlooding
2,3,0.000510,172.27.224.250,172.27.224.251,TCP,60,502 > 54084 [ACK] Seq=1 Ack=2 Win=8712 Len=0,Telemecaniqu_09:51:3b,Telemecaniqu_09:51:3b,Telemecanique Electrique,...,ASUSTek COMPUTER INC.,54084.0,ASUSTekCOMPU_64:40:79,Not set,Set,TCP,1.527317e+09,Not set,0.000510,modbusQueryFlooding
3,4,0.001591,172.27.224.250,172.27.224.251,TCP,60,"502 > 54084 [FIN, ACK] Seq=1 Ack=2 Win=8712 ...",Telemecaniqu_09:51:3b,Telemecaniqu_09:51:3b,Telemecanique Electrique,...,ASUSTek COMPUTER INC.,54084.0,ASUSTekCOMPU_64:40:79,Not set,Set,TCP,1.527317e+09,Not set,0.001591,modbusQueryFlooding
4,5,0.001611,172.27.224.251,172.27.224.250,TCP,60,54084 > 502 [ACK] Seq=2 Ack=2 Win=2036 Len=0,ASUSTekCOMPU_64:40:79,ASUSTekCOMPU_64:40:79,ASUSTek COMPUTER INC.,...,Telemecanique Electrique,502.0,Telemecaniqu_09:51:3b,Not set,Set,TCP,1.527317e+09,Not set,0.001611,modbusQueryFlooding


In [5]:
# Check for missing values
missing_values = modbusQueryFlooding_df.isnull().sum()
print(missing_values)

No.                      0
Time                     0
SrcIP                    0
DstIP                    0
Protocol                 0
Length                   0
Info                     0
SrcMAC                   0
SrcMACResolved           0
SrcOUIResolved           0
SrcPort             376013
SequenceNumber      417962
SrcOUI                   0
DstMAC                   0
DstMACResolved           0
DstOUI                   0
DstOUResolved       168917
DstPort             376013
DstMACResolved.1         0
SYNFlag             417962
ACKFlag             417962
ProtocolType             0
EpochTime                0
TimeDelta           417962
RelativeTime             0
AttackName               0
dtype: int64


In [6]:
# Count the number of packets where TimeDelta should be present but is null
timedelta_missing = modbusQueryFlooding_df[modbusQueryFlooding_df['TimeDelta'].isnull() & ((modbusQueryFlooding_df['Time'].notnull()) & (modbusQueryFlooding_df['EpochTime'].notnull()))].shape[0]
print(f"Number of packets where TimeDelta should be present but is null: {timedelta_missing}")


Number of packets where TimeDelta should be present but is null: 417962


In [8]:
# Sort the DataFrame by EpochTime to ensure the data is in chronological order
modbusQueryFlooding_df.sort_values(by='EpochTime', inplace=True)

# Interpolate missing TimeDelta values based on the EpochTime column
modbusQueryFlooding_df['TimeDelta'] = modbusQueryFlooding_df['EpochTime'].diff().fillna(method='backfill')

# Check if there are any remaining null values in TimeDelta
null_timedelta_count = modbusQueryFlooding_df['TimeDelta'].isnull().sum()
print(f"Number of remaining null values in TimeDelta after interpolation: {null_timedelta_count}")

Number of remaining null values in TimeDelta after interpolation: 0


In [10]:
# 1. Handling missing values in SrcPort and DstPort
modbusQueryFlooding_df['SrcPort'].fillna(-1, inplace=True)  # Filling missing SrcPort with -1
modbusQueryFlooding_df['DstPort'].fillna(-1, inplace=True)  # Filling missing DstPort with -1

# 2. Handling missing values in SequenceNumber
# Assuming TCP packets have SequenceNumber and using median imputation
median_seqnum = modbusQueryFlooding_df[modbusQueryFlooding_df['Protocol'] == 'TCP']['SequenceNumber'].median()
modbusQueryFlooding_df['SequenceNumber'].fillna(median_seqnum, inplace=True)

# 3. Handling missing values in DstOUResolved
modbusQueryFlooding_df['DstOUResolved'].fillna('Unknown', inplace=True)  # Filling missing DstOUResolved with 'Unknown'

# 4. Handling missing values in SYNFlag and ACKFlag
modbusQueryFlooding_df['SYNFlag'].fillna(False, inplace=True)  # Filling missing SYNFlag with False
modbusQueryFlooding_df['ACKFlag'].fillna(False, inplace=True)  # Filling missing ACKFlag with False

In [11]:
# Check for missing values
missing_values = modbusQueryFlooding_df.isnull().sum()
print(missing_values)

No.                 0
Time                0
SrcIP               0
DstIP               0
Protocol            0
Length              0
Info                0
SrcMAC              0
SrcMACResolved      0
SrcOUIResolved      0
SrcPort             0
SequenceNumber      0
SrcOUI              0
DstMAC              0
DstMACResolved      0
DstOUI              0
DstOUResolved       0
DstPort             0
DstMACResolved.1    0
SYNFlag             0
ACKFlag             0
ProtocolType        0
EpochTime           0
TimeDelta           0
RelativeTime        0
AttackName          0
dtype: int64


In [ ]:
# Convert EpochTime to datetime format and create 'date_time' column
modbusQueryFlooding_df['date_time'] = pd.to_datetime(df['EpochTime'], unit='s')

# TcpSYNFlood

In [17]:
csv_files = glob.glob('captures1_v2/tcpSYNFloodDDoS/*.csv')

# Create an empty dataframe to store the combined data
tcpSYNFlood_df = pd.DataFrame()

# Loop through each CSV file and append its contents to the combined dataframe
for csv_file in csv_files:
    try:
        df = pd.read_csv(csv_file)
        tcpSYNFlood_df = pd.concat([tcpSYNFlood_df, df])
    except Exception as e:
        print(f"Failed to read {csv_file}: {e}")

tcpSYNFlood_df['AttackName'] = 'tcpSYNFloodDDoS'

tcpSYNFlood_df.head()

Failed to read captures1_v2/tcpSYNFloodDDoS/eth2dump-tcpSYNFloodDDoS30m-6h_1.csv: 'utf-8' codec can't decode byte 0x86 in position 261916: invalid start byte


,No.,Time,SrcIP,DstIP,Protocol,Length,Info,SrcMAC,SrcMACResolved,SrcOUIResolved,...,DstOUResolved,DstPort,DstMACResolved.1,SYNFlag,ACKFlag,ProtocolType,EpochTime,RelativeTime,TimeDelta,AttackName
0,1,0.000000,172.27.224.251,172.27.224.250,TCP,60,"49659 > 502 [FIN, ACK] Seq=1 Ack=1 Win=2036 ...",48:5b:39:64:40:79,ASUSTekC_64:40:79,ASUSTek COMPUTER INC.,...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,TCP,1.526931e+09,0.000000,0.000000,tcpSYNFloodDDoS
1,2,0.000442,172.27.224.250,172.27.224.251,TCP,60,502 > 49659 [ACK] Seq=1 Ack=2 Win=8712 Len=0,00:80:f4:09:51:3b,Telemech_09:51:3b,Telemechanique Electrique,...,ASUSTek COMPUTER INC.,49659.0,ASUSTekC_64:40:79,Not set,Set,TCP,1.526931e+09,0.000442,0.000442,tcpSYNFloodDDoS
2,3,0.001274,172.27.224.250,172.27.224.251,TCP,60,"502 > 49659 [FIN, ACK] Seq=1 Ack=2 Win=8712 ...",00:80:f4:09:51:3b,Telemech_09:51:3b,Telemechanique Electrique,...,ASUSTek COMPUTER INC.,49659.0,ASUSTekC_64:40:79,Not set,Set,TCP,1.526931e+09,0.001274,0.000832,tcpSYNFloodDDoS
3,4,0.001296,172.27.224.251,172.27.224.250,TCP,60,49659 > 502 [ACK] Seq=2 Ack=2 Win=2036 Len=0,48:5b:39:64:40:79,ASUSTekC_64:40:79,ASUSTek COMPUTER INC.,...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,TCP,1.526931e+09,0.001296,0.000022,tcpSYNFloodDDoS
4,5,0.001472,172.27.224.250,172.27.224.251,TCP,60,"502 > 49659 [RST, ACK] Seq=2 Ack=2 Win=8712 ...",00:80:f4:09:51:3b,Telemech_09:51:3b,Telemechanique Electrique,...,ASUSTek COMPUTER INC.,49659.0,ASUSTekC_64:40:79,Not set,Set,TCP,1.526931e+09,0.001472,0.000176,tcpSYNFloodDDoS


In [18]:
# Check for missing values
missing_values = tcpSYNFlood_df.isnull().sum()
print(missing_values)

No.                      0
Time                     0
SrcIP                    0
DstIP                    0
Protocol                 0
Length                   0
Info                   391
SrcMAC                   0
SrcMACResolved           0
SrcOUIResolved           0
SrcPort             348148
SequenceNumber      389327
SrcOUI                   0
DstMAC                   0
DstMACResolved           0
DstOUI                   0
DstOUResolved       291851
DstPort             348148
DstMACResolved.1         0
SYNFlag             389327
ACKFlag             389327
ProtocolType             0
EpochTime                0
RelativeTime             0
TimeDelta                0
AttackName               0
dtype: int64


In [ ]:
# 1. Handling missing values in SrcPort and DstPort
tcpSYNFlood_df['SrcPort'].fillna(-1, inplace=True)  # Filling missing SrcPort with -1
tcpSYNFlood_df['DstPort'].fillna(-1, inplace=True)  # Filling missing DstPort with -1

# 2. Handling missing values in SequenceNumber
# Assuming TCP packets have SequenceNumber and using median imputation
median_seqnum = tcpSYNFlood_df[tcpSYNFlood_df['Protocol'] == 'TCP']['SequenceNumber'].median()
tcpSYNFlood_df['SequenceNumber'].fillna(median_seqnum, inplace=True)

# 3. Handling missing values in DstOUResolved
tcpSYNFlood_df['DstOUResolved'].fillna('Unknown', inplace=True)  # Filling missing DstOUResolved with 'Unknown'

# 4. Handling missing values in SYNFlag and ACKFlag
tcpSYNFlood_df['SYNFlag'].fillna(False, inplace=True)  # Filling missing SYNFlag with False
tcpSYNFlood_df['ACKFlag'].fillna(False, inplace=True)  # Filling missing ACKFlag with False

# Fill missing Info values with a placeholder like "Unknown"
df['Info'].fillna('Unknown', inplace=True)

: 

In [16]:
# Check for missing values
missing_values = tcpSYNFlood_df.isnull().sum()
print(missing_values)

NameError: name 'tcpSYNFlood_df' is not defined

In [ ]:
# Convert EpochTime to datetime format and create 'date_time' column
tcpSYNFlood_df['date_time'] = pd.to_datetime(df['EpochTime'], unit='s')

# pingFloodDDoS

In [22]:
csv_files = glob.glob('captures1_v2/pingFloodDDoS/*.csv')

pingFloodDDos_df = pd.DataFrame()

for csv_file in csv_files:
    try:
        df = pd.read_csv(csv_file)
        pingFloodDDos_df = pd.concat([pingFloodDDos_df, df])
    except Exception as e:
        print(f"Failed to read {csv_file}: {e}")

pingFloodDDos_df['AttackName'] = 'pingFloodDDoS'

pingFloodDDos_df.head()

,No.,Time,SrcIP,DstIP,Protocol,Length,Info,SrcMAC,SrcMACResolved,SrcOUIResolved,...,DstOUResolved,DstPort,DstMACResolved.1,SYNFlag,ACKFlag,ProtocolType,EpochTime,RelativeTime,TimeDelta,AttackName
0,1,0.000000,172.27.224.251,172.27.224.250,TCP,60,"51152 > 502 [FIN, ACK] Seq=1 Ack=1 Win=2036 ...",48:5b:39:64:40:79,ASUSTekC_64:40:79,ASUSTek COMPUTER INC.,...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,TCP,1.526899e+09,0.000000,0.000000,pingFloodDDoS
1,2,0.009869,172.27.224.250,172.27.224.251,TCP,60,502 > 51152 [ACK] Seq=1 Ack=2 Win=8712 Len=0,00:80:f4:09:51:3b,Telemech_09:51:3b,Telemechanique Electrique,...,ASUSTek COMPUTER INC.,51152.0,ASUSTekC_64:40:79,Not set,Set,TCP,1.526899e+09,0.009869,0.009869,pingFloodDDoS
2,3,0.010712,172.27.224.250,172.27.224.251,TCP,60,"502 > 51152 [FIN, ACK] Seq=1 Ack=2 Win=8712 ...",00:80:f4:09:51:3b,Telemech_09:51:3b,Telemechanique Electrique,...,ASUSTek COMPUTER INC.,51152.0,ASUSTekC_64:40:79,Not set,Set,TCP,1.526899e+09,0.010712,0.000843,pingFloodDDoS
3,4,0.010731,172.27.224.251,172.27.224.250,TCP,60,51152 > 502 [ACK] Seq=2 Ack=2 Win=2036 Len=0,48:5b:39:64:40:79,ASUSTekC_64:40:79,ASUSTek COMPUTER INC.,...,Telemechanique Electrique,502.0,Telemech_09:51:3b,Not set,Set,TCP,1.526899e+09,0.010731,0.000019,pingFloodDDoS
4,5,0.010909,172.27.224.250,172.27.224.251,TCP,60,"502 > 51152 [RST, ACK] Seq=2 Ack=2 Win=8712 ...",00:80:f4:09:51:3b,Telemech_09:51:3b,Telemechanique Electrique,...,ASUSTek COMPUTER INC.,51152.0,ASUSTekC_64:40:79,Not set,Set,TCP,1.526899e+09,0.010909,0.000178,pingFloodDDoS


In [26]:
# Check for missing values
missing_values = pingFloodDDos_df.isnull().sum()
print(missing_values)

No.                       0
Time                      0
SrcIP                     0
DstIP                     0
Protocol                  0
Length                    0
Info                      0
SrcMAC                    0
SrcMACResolved            0
SrcOUIResolved            0
SrcPort             1803759
SequenceNumber      1845741
SrcOUI                    0
DstMAC                    0
DstMACResolved            0
DstOUI                    0
DstOUResolved        315586
DstPort             1803759
DstMACResolved.1          0
SYNFlag             1845741
ACKFlag             1845741
ProtocolType              0
EpochTime                 0
RelativeTime              0
TimeDelta                 0
AttackName                0
dtype: int64


In [ ]:
# 1. Handling missing values in SrcPort and DstPort
pingFloodDDos_df['SrcPort'].fillna(-1, inplace=True)  # Filling missing SrcPort with -1
pingFloodDDos_df['DstPort'].fillna(-1, inplace=True)  # Filling missing DstPort with -1

# 2. Handling missing values in SequenceNumber
# Assuming TCP packets have SequenceNumber and using median imputation
median_seqnum = pingFloodDDos_df[pingFloodDDos_df['Protocol'] == 'TCP']['SequenceNumber'].median()
pingFloodDDos_df['SequenceNumber'].fillna(median_seqnum, inplace=True)

# 3. Handling missing values in DstOUResolved
pingFloodDDos_df['DstOUResolved'].fillna('Unknown', inplace=True)  # Filling missing DstOUResolved with 'Unknown'

# 4. Handling missing values in SYNFlag and ACKFlag
pingFloodDDos_df['SYNFlag'].fillna(False, inplace=True)  # Filling missing SYNFlag with False
pingFloodDDos_df['ACKFlag'].fillna(False, inplace=True)  # Filling missing ACKFlag with False


In [ ]:
# Check for missing values
missing_values = pingFloodDDos_df.isnull().sum()
print(missing_values)

In [ ]:
# Convert EpochTime to datetime format and create 'date_time' column
pingFloodDDos_df['date_time'] = pd.to_datetime(df['EpochTime'], unit='s')